### 使用经过lora微调的llava模型

In [4]:
from transformers import AutoModelForCausalLM, AutoProcessor, LlavaForConditionalGeneration 
from PIL import Image
from peft import PeftModel
import torch
import os

In [5]:
device = "cuda:0"
# 基础模型路径（ lora 微调的初始模型）
base_model_path = "./qwen2.5_3B_Instruct_clipvL14_model/model001"
# lora 微调模型路径
lora_model_path = "./output_model_lora_show/[epoch4-5]qwen2.5_3B_Instruct_clipvL14"

#### 载入基础模型与[epoch4-5]qwen2.5_3B_Instruct_clipvL14的lora微调数据，调试并以完整参数格式保存模型

In [ ]:
# 加载 base 基础模型
model = LlavaForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path=base_model_path,
    device_map=device,
    torch_dtype=torch.bfloat16
)

# 加载训练得到的 lora 适配器
model = PeftModel.from_pretrained(model, lora_model_path)

# 加载 processor
processor = AutoProcessor.from_pretrained(pretrained_model_name_or_path=base_model_path)

In [ ]:
print(processor)

In [ ]:
prompt = "USER: <image>\nWhat's the content of the image? Describe in 88 words. ASSISTANT:"
url = "data/australia.jpg"
image = Image.open(fp=url)

inputs = processor(images=image, text=prompt, return_tensors="pt")

for temp_key in inputs.keys():
    inputs[temp_key] = inputs[temp_key].to(device)

In [ ]:
# Generate
generate_ids = model.generate(**inputs, max_new_tokens=200)
# 将模型生成的文本 token 序列 解码成可读的字符串形式
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
response